# Supervised Learning - Regression and hyperparameter tuning

In [17]:
%pip install pandas
%pip install StandardScaler            
%pip install matplotlib 
%pip install seaborn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import CSV File

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    cohen_kappa_score,
    roc_auc_score,
    classification_report,
)

df = pd.read_csv('data/cox2.csv')  


## Split the dataset

In [2]:
X = df.drop(columns=['cox2Class', 'IC50'])  
y = df['cox2Class'] 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
) 

## Select a Learning Method & Preprocessing

In [ ]:
scaler = StandardScaler()  
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)

rf = RandomForestClassifier(random_state=42)  


## Define a tuning grid

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2']
}  

grid_search = GridSearchCV(
    rf,
    param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)  

4. Beste Parameter: {'max_depth': 10, 'max_features': 'log2', 'n_estimators': 100}
   Bestes CV-MSE: 0.2636393732327271


## Analyze performance values

In [22]:
best_rf = grid_search.best_estimator_
importances = best_rf.feature_importances_
feature_names = X.columns
sorted_idx = importances.argsort()[::-1]

print("\n6. Feature Importances (Top 10):")
for i in range(10):
    print(f"   {feature_names[sorted_idx[i]]}: {importances[sorted_idx[i]]:.4f}")


6. Feature Importances (Top 10):
   tcnp: 0.0346
   tpsa: 0.0336
   polar_area: 0.0308
   most_positive_charge: 0.0291
   logp.o.w.: 0.0283
   vsa_acc: 0.0253
   fnsa3: 0.0252
   psa_npsa: 0.0239
   prx: 0.0206
   mlogp: 0.0196


## Apply the final model to the test set

In [23]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = best_rf.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n7. Test-MSE: {mse:.4f}")
print(f"   Test-R2: {r2:.4f}")


7. Test-MSE: 0.2496
   Test-R2: 0.4029
